In [1]:
# Slighlty updated for Keras 2.0 APIs
# - by Kyeong Soo (Joseph) Kim

import datetime
import os
import numpy as np
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL']='2' # supress warning messages
import tensorflow as tf

from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.preprocessing import scale
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
path_train = "../data/UJIIndoorLoc/trainingData2.csv"
path_validation = "../data/UJIIndoorLoc/validationData2.csv"
path_results = "../results/indoor_localization-1" # TODO: implement auto-detection of the file name

In [3]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.iloc[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths).astype(float), axis=1) # convert integer to float and scale jointly (axis=1)

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

In [4]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [5]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [6]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]).astype(float))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [7]:
epochs = 20
batch_size = 10
input_size = 520
num_classes = 13
# nums_hidden_nodes = [16] # for test
nums_hidden_nodes = [16, 32, 64, 128, 256, 512]
# dropout_rates = [0.5] # for test
dropout_rates = [0.0, 0.2, 0.5]
# dropout_rates = np.arange(11)*0.05 # 0.0, 0.05,...,0.5
losses = np.zeros((len(nums_hidden_nodes), len(dropout_rates)))
accuracies = np.zeros_like(losses)

In [8]:
def classifier(num_hidden_nodes, dropout_rate):
    model = Sequential()
    model.add(Dense(num_hidden_nodes, input_dim=input_size, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(256, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(128, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(64, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax', use_bias=True))
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [9]:
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        c = classifier(nums_hidden_nodes[nh], dropout_rates[dr])
        startTime = timer()
        c.fit(train_X, train_y, validation_data=(val_X, val_y), batch_size=batch_size, epochs=epochs)
        elapsedTime = timer() - startTime
        print("Model trained with nh=%d & dr=%.2f in %e s.\n" % (nums_hidden_nodes[nh], dropout_rates[dr], elapsedTime))
        loss, acc = c.evaluate(test_AP_features, test_labels)
        losses[nh, dr] = loss
        accuracies[nh, dr] = acc

Train on 13888 samples, validate on 6042 samples
Epoch 1/20
13888/13888 [==============================] - 8s 580us/step - loss: 0.4183 - acc: 0.9155 - val_loss: 0.0857 - val_acc: 0.9873
Epoch 2/20
13888/13888 [==============================] - 6s 403us/step - loss: 0.0564 - acc: 0.9905 - val_loss: 0.0415 - val_acc: 0.9907
Epoch 3/20
13888/13888 [==============================] - 6s 398us/step - loss: 0.0313 - acc: 0.9930 - val_loss: 0.0347 - val_acc: 0.9902
Epoch 4/20
13888/13888 [==============================] - 5s 380us/step - loss: 0.0200 - acc: 0.9945 - val_loss: 0.0261 - val_acc: 0.9914
Epoch 5/20
13888/13888 [==============================] - 5s 389us/step - loss: 0.0164 - acc: 0.9943 - val_loss: 0.0346 - val_acc: 0.9879
Epoch 6/20
13888/13888 [==============================] - 5s 390us/step - loss: 0.0134 - acc: 0.9955 - val_loss: 0.0257 - val_acc: 0.9916
Epoch 7/20
13888/13888 [==============================] - 5s 394us/step - loss: 0.0116 - acc: 0.9958 - val_loss: 0.0251 - v

13888/13888 [==============================] - 6s 400us/step - loss: 0.0057 - acc: 0.9972 - val_loss: 0.0266 - val_acc: 0.9909
Epoch 18/20
13888/13888 [==============================] - 6s 396us/step - loss: 0.0060 - acc: 0.9970 - val_loss: 0.0292 - val_acc: 0.9922
Epoch 19/20
13888/13888 [==============================] - 6s 432us/step - loss: 0.0066 - acc: 0.9968 - val_loss: 0.0278 - val_acc: 0.9922
Epoch 20/20
13888/13888 [==============================] - 6s 434us/step - loss: 0.0054 - acc: 0.9979 - val_loss: 0.0330 - val_acc: 0.9911
Model trained with nh=16 & dr=0.50 in 1.151066e+02 s.

1111/1111 [==============================] - 0s 51us/step
Train on 13888 samples, validate on 6042 samples
Epoch 1/20
13888/13888 [==============================] - 6s 452us/step - loss: 0.2563 - acc: 0.9415 - val_loss: 0.0526 - val_acc: 0.9899
Epoch 2/20
13888/13888 [==============================] - 6s 425us/step - loss: 0.0376 - acc: 0.9908 - val_loss: 0.0300 - val_acc: 0.9916
Epoch 3/20
13888/1

13888/13888 [==============================] - 10s 709us/step - loss: 0.0072 - acc: 0.9975 - val_loss: 0.0272 - val_acc: 0.9921
Epoch 13/20
13888/13888 [==============================] - 10s 749us/step - loss: 0.0063 - acc: 0.9969 - val_loss: 0.0262 - val_acc: 0.9916
Epoch 14/20
13888/13888 [==============================] - 9s 675us/step - loss: 0.0053 - acc: 0.9975 - val_loss: 0.0217 - val_acc: 0.9934
Epoch 15/20
13888/13888 [==============================] - 10s 691us/step - loss: 0.0067 - acc: 0.9972 - val_loss: 0.0275 - val_acc: 0.9917
Epoch 16/20
13888/13888 [==============================] - 9s 644us/step - loss: 0.0055 - acc: 0.9972 - val_loss: 0.0227 - val_acc: 0.9927
Epoch 17/20
13888/13888 [==============================] - 9s 650us/step - loss: 0.0046 - acc: 0.9976 - val_loss: 0.0244 - val_acc: 0.9926
Epoch 18/20
13888/13888 [==============================] - 9s 663us/step - loss: 0.0049 - acc: 0.9976 - val_loss: 0.0263 - val_acc: 0.9921
Epoch 19/20
13888/13888 [===========

13888/13888 [==============================] - 10s 751us/step - loss: 0.0075 - acc: 0.9971 - val_loss: 0.0205 - val_acc: 0.9935
Epoch 8/20
13888/13888 [==============================] - 10s 721us/step - loss: 0.0083 - acc: 0.9966 - val_loss: 0.0245 - val_acc: 0.9921
Epoch 9/20
13888/13888 [==============================] - 11s 796us/step - loss: 0.0079 - acc: 0.9966 - val_loss: 0.0221 - val_acc: 0.9934
Epoch 10/20
13888/13888 [==============================] - 10s 750us/step - loss: 0.0059 - acc: 0.9976 - val_loss: 0.0204 - val_acc: 0.9934
Epoch 11/20
13888/13888 [==============================] - 12s 899us/step - loss: 0.0053 - acc: 0.9976 - val_loss: 0.0253 - val_acc: 0.9926
Epoch 12/20
13888/13888 [==============================] - 11s 768us/step - loss: 0.0079 - acc: 0.9964 - val_loss: 0.0215 - val_acc: 0.9924
Epoch 13/20
13888/13888 [==============================] - 11s 765us/step - loss: 0.0060 - acc: 0.9970 - val_loss: 0.0280 - val_acc: 0.9924
Epoch 14/20
13888/13888 [=========

13888/13888 [==============================] - 10s 736us/step - loss: 0.0240 - acc: 0.9928 - val_loss: 0.0194 - val_acc: 0.9934
Epoch 3/20
13888/13888 [==============================] - 10s 755us/step - loss: 0.0157 - acc: 0.9942 - val_loss: 0.0244 - val_acc: 0.9916
Epoch 4/20
13888/13888 [==============================] - 10s 754us/step - loss: 0.0140 - acc: 0.9948 - val_loss: 0.0210 - val_acc: 0.9926
Epoch 5/20
13888/13888 [==============================] - 10s 740us/step - loss: 0.0127 - acc: 0.9955 - val_loss: 0.0215 - val_acc: 0.9924
Epoch 6/20
13888/13888 [==============================] - 11s 766us/step - loss: 0.0095 - acc: 0.9965 - val_loss: 0.0218 - val_acc: 0.9924
Epoch 7/20
13888/13888 [==============================] - 11s 794us/step - loss: 0.0096 - acc: 0.9956 - val_loss: 0.0230 - val_acc: 0.9926
Epoch 8/20
13888/13888 [==============================] - 10s 747us/step - loss: 0.0095 - acc: 0.9959 - val_loss: 0.0192 - val_acc: 0.9939
Epoch 9/20
13888/13888 [==============

13888/13888 [==============================] - 24s 2ms/step - loss: 0.0057 - acc: 0.9974 - val_loss: 0.0187 - val_acc: 0.9944
Epoch 19/20
13888/13888 [==============================] - 11s 786us/step - loss: 0.0054 - acc: 0.9976 - val_loss: 0.0222 - val_acc: 0.9945
Epoch 20/20
13888/13888 [==============================] - 10s 715us/step - loss: 0.0065 - acc: 0.9973 - val_loss: 0.0177 - val_acc: 0.9949
Model trained with nh=256 & dr=0.20 in 2.779035e+02 s.

1111/1111 [==============================] - 0s 88us/step
Train on 13888 samples, validate on 6042 samples
Epoch 1/20
13888/13888 [==============================] - 11s 787us/step - loss: 0.1045 - acc: 0.9703 - val_loss: 0.0395 - val_acc: 0.9874
Epoch 2/20
13888/13888 [==============================] - 11s 772us/step - loss: 0.0296 - acc: 0.9898 - val_loss: 0.0533 - val_acc: 0.9811
Epoch 3/20
13888/13888 [==============================] - 10s 738us/step - loss: 0.0183 - acc: 0.9935 - val_loss: 0.0401 - val_acc: 0.9846
Epoch 4/20
138

13888/13888 [==============================] - 6s 411us/step - loss: 0.0096 - acc: 0.9963 - val_loss: 0.0623 - val_acc: 0.9861
Epoch 14/20
13888/13888 [==============================] - 6s 410us/step - loss: 0.0164 - acc: 0.9950 - val_loss: 0.0333 - val_acc: 0.9922
Epoch 15/20
13888/13888 [==============================] - 6s 420us/step - loss: 0.0077 - acc: 0.9968 - val_loss: 0.0289 - val_acc: 0.9934
Epoch 16/20
13888/13888 [==============================] - 6s 400us/step - loss: 0.0073 - acc: 0.9970 - val_loss: 0.0706 - val_acc: 0.9846
Epoch 17/20
13888/13888 [==============================] - 6s 423us/step - loss: 0.0069 - acc: 0.9973 - val_loss: 0.0266 - val_acc: 0.9944
Epoch 18/20
13888/13888 [==============================] - 6s 425us/step - loss: 0.0079 - acc: 0.9976 - val_loss: 0.0257 - val_acc: 0.9945
Epoch 19/20
13888/13888 [==============================] - 6s 406us/step - loss: 0.0125 - acc: 0.9966 - val_loss: 0.0479 - val_acc: 0.9894
Epoch 20/20
13888/13888 [==============

In [11]:
now = datetime.datetime.now()
path_results += "_" + now.strftime("%Y%m%d-%H%M%S") + ".csv"
f = open(path_results, 'w')
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        f.write("nh=%d & dr=%.2f: loss=%e, accuracy=%e" % (nums_hidden_nodes[nh], dropout_rates[dr], losses[nh, dr], accuracies[nh, dr]))
f.close()